In [123]:
import pandas as pd

In [138]:
# Conectando e baixando arquivos do GitHub

arquivos = ["DadosBase2023.csv", "DadosBase2024.csv", "SH.csv", "NomesMunicipios.csv", "Paises.csv",]
url_base = "https://raw.githubusercontent.com/Lucasdz-7/Projeto-Integrador-1/f81f64f9a0324de118ead77d63aba9e33bb98e32/Projeto/"


In [139]:
# Criando Dataframes

Exportacao23 = pd.read_csv(url_base + 'DadosBase2023.csv', low_memory=False, sep=';', encoding='UTF-8')
Exportacao24 = pd.read_csv(url_base + 'DadosBase2024.csv', low_memory=False, sep=';', encoding='UTF-8')
SH = pd.read_csv(url_base + 'SH.csv', low_memory=False, sep=';', encoding='ISO-8859-1')
Municipios = pd.read_csv(url_base + 'NomesMunicipios.csv', low_memory=False, sep=';', encoding='ISO-8859-1')

In [140]:
# Juntando dados de exportação

ExportacaoTotal = pd.concat([Exportacao23, Exportacao24],ignore_index=False)
ExportacaoTotal.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2266180 entries, 0 to 1144827
Data columns (total 8 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   CO_ANO      int64 
 1   CO_MES      int64 
 2   SH4         int64 
 3   CO_PAIS     int64 
 4   SG_UF_MUN   object
 5   CO_MUN      int64 
 6   KG_LIQUIDO  int64 
 7   VL_FOB      int64 
dtypes: int64(7), object(1)
memory usage: 155.6+ MB


In [96]:
ExportacaoTotal.head(5)

,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB
0,2023,10,2209,127,BA,2919553,36,53
1,2023,11,4009,493,SP,3456701,473,14008
2,2023,11,8536,365,SP,3438709,2,181
3,2023,8,8512,63,RJ,3304201,15627,239105
4,2023,6,1902,580,PR,4118204,287,368


In [97]:
Municipios.head(10)

,CO_MUN_GEO,NO_MUN,NO_MUN_MIN,SG_UF
0,1506807,SANTAREM,Santarém,PA
1,1506906,SANTAREM NOVO,Santarém Novo,PA
2,1507003,SANTO ANTONIO DO TAUA,Santo Antônio do Tauá,PA
3,1507102,SAO CAETANO DE ODIVELAS,São Caetano de Odivelas,PA
4,1507151,SAO DOMINGOS DO ARAGUAIA,São Domingos do Araguaia,PA
5,1507201,SAO DOMINGOS DO CAPIM,São Domingos do Capim,PA
6,1507300,SAO FELIX DO XINGU,São Félix do Xingu,PA
7,1507409,SAO FRANCISCO DO PARA,São Francisco do Pará,PA
8,1507458,SAO GERALDO DO ARAGUAIA,São Geraldo do Araguaia,PA
9,1507466,SAO JOAO DA PONTA,São João da Ponta,PA


In [141]:
# Tratando os dados do arquivo de municípios
MunicipiosSp = Municipios.query("SG_UF == 'SP'")
MunicipiosSp.head(10)
MunicipiosSp.rename(columns={"CO_MUN_GEO": "CO_MUN"}, inplace=True)

<ipython-input-141-6a5274dcace2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MunicipiosSp.rename(columns={"CO_MUN_GEO": "CO_MUN"}, inplace=True)


In [142]:
# Tratando dados do arquivo de Exportação para visualização apenas do estado de São Paulo

ExportacaoSp = ExportacaoTotal.query("SG_UF_MUN == 'SP'")
ExportacaoSp.head(5)

,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB
1,2023,11,4009,493,SP,3456701,473,14008
2,2023,11,8536,365,SP,3438709,2,181
5,2023,6,8421,63,SP,3450308,6002,131714
6,2023,4,9403,586,SP,3447304,70,264
9,2023,6,8419,158,SP,3409502,44,4456


In [143]:
# Juntando arquivos de municípios e de Exportação

ExportacaoSp = ExportacaoSp.merge(MunicipiosSp[['CO_MUN','NO_MUN_MIN']], on = 'CO_MUN', how = 'left')
ExportacaoSp.head(5)

,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB,NO_MUN_MIN
0,2023,11,4009,493,SP,3456701,473,14008,Vinhedo
1,2023,11,8536,365,SP,3438709,2,181,Piracicaba
2,2023,6,8421,63,SP,3450308,6002,131714,São Paulo
3,2023,4,9403,586,SP,3447304,70,264,Santana de Parnaíba
4,2023,6,8419,158,SP,3409502,44,4456,Campinas


In [145]:
# Renomeando colunas

ExportacaoSp.drop(columns=['CO_MUN'], inplace=True)
ExportacaoSp.rename(columns={'NO_MUN_MIN': 'Nome_Municipio'}, inplace=True)
ExportacaoSp.head(10)

KeyError: "['CO_MUN'] not found in axis"

In [131]:
SH.head(5)

,CO_SH6,NO_SH6_POR,NO_SH6_ESP,NO_SH6_ING,CO_SH4,NO_SH4_POR,NO_SH4_ESP,NO_SH4_ING,CO_SH2,NO_SH2_POR,NO_SH2_ESP,NO_SH2_ING,CO_NCM_SECROM,NO_SEC_POR,NO_SEC_ESP,NO_SEC_ING
0,10110,"Animais vivos das espécies cavalar, asinina e ...","Caballos y asnos, reproductores de raza pura",Pure-bred breeding horses and asses,101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products
1,10111,"Cavalos reprodutores, de raça pura",Caballos reproductores de raza pura,Pure-bred breeding horses,101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products
2,10119,"Outros cavalos, vivos","Demás caballos, vivos",Other live horses,101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products
3,10120,Asininos e muares vivos,"Asnos, mulos y burdéganos, vivos","Asses, mules and hinnies (live)",101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products
4,10121,Cavalos reprodutores de raça pura,Caballos de raza selecta para reproducción,Pure-bred breeding horses,101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products


In [146]:
SH.drop(columns=['CO_SH6', 'NO_SH6_POR',	'NO_SH6_ESP',	'NO_SH6_ING', 'NO_SH4_ESP', 'NO_SH4_ING',
'CO_SH2',	'NO_SH2_POR',	'NO_SH2_ESP',	'NO_SH2_ING',	'CO_NCM_SECROM',	'NO_SEC_POR',
'NO_SEC_ESP',	'NO_SEC_ING'], inplace=True)
SH.rename(columns={'CO_SH4': 'SH4'}, inplace=True)
SH = SH.drop_duplicates(subset='SH4')


SH.head(5)

,SH4,NO_SH4_POR
0,101,"Cavalos, asininos e muares, vivos"
8,102,Animais vivos da espécie bovina
14,103,Animais vivos da espécie suína
17,104,Animais vivos das espécies ovina e caprina
19,105,"Galos, galinhas, patos, gansos, perus, peruas ..."


In [148]:
ExportacaoSp = ExportacaoSp.merge(SH[['SH4','NO_SH4_POR']], on = 'SH4', how = 'left')
ExportacaoSp.head(5)

,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,KG_LIQUIDO,VL_FOB,Nome_Municipio,NO_SH4_POR
0,2023,11,4009,493,SP,473,14008,Vinhedo,"Tubos de borracha vulcanizada não endurecida, ..."
1,2023,11,8536,365,SP,2,181,Piracicaba,"Aparelhos para interrupção, seccionamento, pro..."
2,2023,6,8421,63,SP,6002,131714,São Paulo,"Centrifugadores, incluídos os secadores centrí..."
3,2023,4,9403,586,SP,70,264,Santana de Parnaíba,Outros móveis e suas partes
4,2023,6,8419,158,SP,44,4456,Campinas,"Aparelhos e dispositivos, mesmo aquecidos elec..."
